In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
import re
import requests
from dotenv import load_dotenv



In [4]:
DATA_PATH = "C:\\Users\\MSI\\Desktop\\projet_ml\\customer_review_analysis\\data\\cleaned\\cleaned_reviews_general.csv"
if not os.path.exists(DATA_PATH):
    print(f"Error: File not found at {DATA_PATH}")
else:
    # Load the data
    df = pd.read_csv(DATA_PATH, encoding='utf-8-sig')
    print(f"Successfully loaded {len(df):,} reviews")

Successfully loaded 560 reviews


In [3]:
print("MISSING VALUES")

# For rating column, count NaN values + 0 values as missing
# For other columns, count only NaN values
missing_counts = {}

for column in df.columns:
    if column == 'rating':
        # Count NaN + 0 values for rating
        nan_count = df[column].isna().sum()
        zero_count = (df[column] == 0).sum()
        missing_counts[column] = nan_count + zero_count
    else:
        # Count only NaN for other columns
        missing_counts[column] = df[column].isna().sum()

# Convert to Series
missing = pd.Series(missing_counts)
missing_percent = (missing / len(df)) * 100

missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing': missing.values,
    'Percent': missing_percent.values
})

print(missing_df.to_string(index=False))

MISSING VALUES
  Column  Missing   Percent
      id        0  0.000000
    name        0  0.000000
  source        0  0.000000
location        0  0.000000
    date        0  0.000000
  rating        0  0.000000
 comment      604 52.705061
